## 1. Load and Clean the Corruption Dataset

We start by loading the political corruption index dataset, which includes corruption scores by country and year. However, several country codes are missing or inconsistent. We manually assign ISO 3-letter codes to these non-standard regions using a custom mapping dictionary.

This standardization ensures that every country in this dataset can later be matched correctly with GDP and health data.


In [1]:
import pandas as pd
import pycountry
corruption_df = pd.read_csv("../Data/political-corruption-index.csv")


code_mapping = {
    "Africa": "AFR",
    "Asia": "ASI",
    "Brunswick": "BRW",
    "Democratic Republic of Vietnam": "DRV",
    "Duchy of Nassau": "NAS",
    "Europe": "EUR",
    "North America": "NAM",
    "Oceania": "OCN",
    "Oldenburg": "OLD",
    "Palestine/Gaza": "PGA",
    "Palestine/West Bank": "PWB",
    "Piedmont-Sardinia": "PMS",
    "Saxe-Weimar-Eisenach": "SWE",
    "South America": "SAM",
    "Wurttemberg": "WUR"
}


valid_gdp_codes = [
    "ARG", "AUS", "AUT", "BEL", "BRA", "BGR", "CAN", "CHL", "CHN", "COL", "CRI", "HRV", "CYP", "CZE", "DNK", "EST",
    "FIN", "FRA", "DEU", "GRC", "HUN", "ISL", "IND", "IDN", "IRL", "ISR", "ITA", "JPN", "LVA", "LTU", "LUX", "MLT",
    "MEX", "NLD", "NZL", "NOR", "PER", "POL", "PRT", "ROU", "SVK", "SVN", "ZAF", "KOR", "ESP", "SWE", "CHE", "TUR",
    "UKR", "GBR", "USA"
]

corruption_df['Code'] = corruption_df.apply(
    lambda row: code_mapping.get(row['Entity'], row['Code']) 
    if pd.isna(row['Code']) or str(row['Code']).strip() == '' else row['Code'],
    axis=1
)


corruption_filtered = corruption_df[corruption_df['Code'].isin(valid_gdp_codes)]


corruption_filtered.to_csv("../Temporary Files/corruption_filtered_matched_to_gdp.csv", index=False)

print("Filtered and processed corruption dataset saved as 'corruption_filtered_matched_to_gdp.csv'")


Filtered and processed corruption dataset saved as 'corruption_filtered_matched_to_gdp.csv'


## 2. Preview Cleaned Corruption Dataset

We load the cleaned version of the corruption dataset to visually inspect the assigned codes and confirm that the data is ready for merging.

This step helps verify that our mapping logic worked as expected.


In [2]:
corruption_filtered = pd.read_csv("../Temporary Files/corruption_filtered_matched_to_gdp.csv")
corruption_filtered.head()

,Entity,Code,Year,"Political corruption index (central estimate, aggregate: average)"
0,Argentina,ARG,1789,0.321
1,Argentina,ARG,1790,0.321
2,Argentina,ARG,1791,0.321
3,Argentina,ARG,1792,0.321
4,Argentina,ARG,1793,0.321


## 3. Merge GDP and Corruption Data

We merge the public health expenditure dataset with the corruption dataset using common keys: `Code` and `Year`.

Only records from the years 2010 to 2021 are retained, as they are more relevant and consistent in terms of data availability and global comparability. We also clean up redundant columns and rename them for clarity.


In [3]:
gdp_health_df = pd.read_csv("../Data/Percentage of GDP spemt on Public Health.csv")
corruption_df = pd.read_csv("../Data/political-corruption-index.csv")

merged_df = pd.merge(
    gdp_health_df,
    corruption_df,
    on=['Code', 'Year'],
    how='inner',
    suffixes=('_gdp', '_corruption')
)


merged_df = merged_df[(merged_df['Year'] >= 2010) & (merged_df['Year'] <= 2021)]


merged_df.drop(columns=['Entity_corruption'], inplace=True)
merged_df.rename(columns={'Entity_gdp': 'Entity'}, inplace=True)


merged_df.to_csv("../Temporary Files/merged_gdp_corruption_2010_2021.csv", index=False)

print("Final dataset saved as 'merged_gdp_corruption_2010_2021.csv'")


Final dataset saved as 'merged_gdp_corruption_2010_2021.csv'


## 4. Filter IHME Health Data to Match Available Countries

The IHME dataset contains detailed global health data by cause of death, age, and sex for various countries. However, it uses country names instead of codes.

To align with our merged GDP-corruption dataset, we:
- Extract unique country names from IHME
- Convert them into ISO codes using `pycountry`
- Filter out only those countries for which we have valid GDP and corruption data

This ensures consistency across all three datasets.


In [4]:
df = pd.read_csv("../Data/IHME-GBD_2021_DATA-cc52e0a9-1.csv")
df2 = pd.read_csv("../Temporary Files/merged_gdp_corruption_2010_2021.csv")


unique_countries = df['location_name'].unique()


def get_country_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

country_codes_df = pd.DataFrame(unique_countries, columns=["Country Name"])
country_codes_df["Country Code"] = country_codes_df["Country Name"].apply(get_country_code)


valid_codes = df2['Code'].dropna().unique()
matched_df = country_codes_df[country_codes_df["Country Code"].isin(valid_codes)]


filtered_df = df[df['location_name'].isin(matched_df["Country Name"])].copy()
country_code_mapping = dict(zip(matched_df['Country Name'], matched_df['Country Code']))
filtered_df['Code'] = filtered_df['location_name'].map(country_code_mapping)


filtered_df.to_csv("../Temporary Files/filtered_dataset_with_codes.csv", index=False)


## 5. Merge Health Data with GDP and Corruption Indicators

We merge the filtered IHME dataset with the combined GDP-corruption dataset on country code and year.

The result is a comprehensive dataset containing:
- Health outcomes (percent or rate of disease burden)
- Public healthcare investment
- Corruption index

This forms the base dataset for any downstream statistical or ML analysis.


In [5]:
filtered_df = pd.read_csv("../Temporary Files/filtered_dataset_with_codes.csv")
gdp_corruption_df = pd.read_csv("../Temporary Files/merged_gdp_corruption_2010_2021.csv")
gdp_corruption_df.rename(columns={"Year": "year"}, inplace=True)
merged_df = pd.merge(filtered_df, gdp_corruption_df, on=["Code", "year"], how="inner")
merged_df.to_csv("../Temporary Files/final_merged_dataset.csv", index=False)


## 6. Preview Final Merged Dataset

We reload and inspect the final merged dataset saved from the previous step.

This is to confirm that the merging process was successful and the dataset structure is intact.


In [6]:
columns_to_drop = [
    "measure_id", "measure_name", "location_id",
    "sex_id", "sex_name", "age_id", "age_name",
    "metric_id", "Entity"
]
cleaned_df = merged_df.drop(columns=columns_to_drop)
cleaned_df.to_csv("../Temporary Files/cleaned_final_merged_dataset.csv", index=False)


## 7. Remove Irrelevant Metric Values

The IHME dataset includes multiple metrics such as "Number", "Percent", and "Rate". For our analysis, we only retain "Percent" and "Rate", since "Number" is not normalized and could skew the results.

This step filters out those rows where metric is "Number".


In [7]:
cleaned_df = pd.read_csv("../Temporary Files/cleaned_final_merged_dataset.csv")
cleaned_df.columns = [col.strip().lower().replace(" ", "_") for col in cleaned_df.columns]
desired_order = [
    "code", "location_name", "year",
    "public_health_expenditure_as_a_share_of_gdp",
    "political_corruption_index_(central_estimate,_aggregate:_average)",
    "cause_id", "cause_name", "metric_name",
    "val", "upper", "lower"
]
reordered_df = cleaned_df[desired_order]
reordered_df.rename(columns={
    "code": "country_code",
    "location_name": "country",
    "public_health_expenditure_as_a_share_of_gdp": "gdp_health_expenditure",
    "political_corruption_index_(central_estimate,_aggregate:_average)": "political_corruption_index",
    "cause_id": "cause_id",
    "cause_name": "cause_name",
    "metric_name": "metric",
    "val": "value",
    "upper": "upper_value",
    "lower": "lower_value"
}, inplace=True)
reordered_df.to_csv("../Temporary Files/final_ordered_renamed_dataset.csv", index=False)


## 8. Sort and Export Final Cleaned Dataset

Finally, we sort the cleaned dataset by country code and year, ensuring a structured view for time series analysis or unsupervised learning.

This dataset is saved and ready for techniques like PCA, K-means, SVD, or clustering.


In [8]:
df = pd.read_csv("../Temporary Files/final_ordered_renamed_dataset.csv")
filtered_df = df[~df['metric'].str.lower().eq('number')]
filtered_df.columns = [
    "country_code", "country", "year", "gdp_health_expenditure",
    "political_corruption_index", "cause_id", "cause_name",
    "metric", "value", "upper_value", "lower_value"
]
filtered_df = filtered_df.sort_values(by=["country_code", "year"])
filtered_df.to_csv("../Data/filtered_dataset.csv", index=False)
